In [1]:
import torch
from classes import FullFileForTestings  # Import dataset class and model
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from classes import SpeedEstimatorRNNModified

In [2]:
#Hyperparameters
input_size = 26  # Number of CAN signals per timestep
hidden_size = [256, 256, 256, 512, 512, 512]
num_layers = [2, 3, 4, 2, 3, 4]
output_size = 1

In [5]:
num_models = 6
num_test_data_path = [r"C:\my files\thesis\AI_training\data\testing_data\testing_data_noise_extended\auax_drv_noise_30.csv", r"C:\my files\thesis\AI_training\data\testing_data\testing_data_noise_extended\buax_brk_noise_30.csv",
 r"C:\my files\thesis\AI_training\data\testing_data\testing_data_noise_extended\sin_steer_noise_30.csv",
    r"C:\my files\thesis\AI_training\data\testing_data\testing_data_noise_extended\suay_mid_noise_30.csv"]
test_data = ["auax_drv", "buax_brk", "sin_steer", "suay_mid"]
model_location = r"C:\my files\thesis\AI_training\Simple RNN\trained_models\iteration_5\model_"

for i in range(num_models):
    # Set paths and parameters for testing
    model_path = model_location + str(i) + '.pt'  # Replace with the path to your saved model

    for j in range(len(num_test_data_path)):
        test_csv_path = num_test_data_path[j]
        # Prepare the test dataset
        test_dataset = FullFileForTestings(test_csv_path)
        features, actual_speeds = test_dataset.get_full_data()

        # Pass all data to the model directly
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        features = features.unsqueeze(0)  # Add batch dimension
        features = features.to(device)

        # Define and load the model
        model = SpeedEstimatorRNNModified(input_size, hidden_size[i], num_layers[i], output_size)  # Instantiate your RNN model
        checkpoint = torch.load(model_path, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])  # Load the trained weights
        model = model.to(device)  # Move model to the appropriate device
        model.eval()  # Set model to evaluation mode

        # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds.numpy(), label="Actual Speeds")
        plt.plot(predicted_speeds, label="Predicted Speeds", linestyle="dashed")
        plt.legend()
        plt.title("Comparison of Actual vs. Predicted Vehicle Speeds (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed [m/s]")
        plt.grid()
        plt.savefig(r"C:\my files\thesis\AI_training\Simple RNN\testing_graphs\iteration_5\model_" + str(i) + "\model_" + str(i) + "_" +test_data[j] + "_act_vs_predicted_speed.png", dpi=300,)
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds.numpy()-predicted_speeds, label="Difference")
        plt.legend()
        plt.title("Difference of Actual vs. Predicted Vehicle Speeds (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed difference [m/s]")
        plt.grid()
        plt.savefig(r"C:\my files\thesis\AI_training\Simple RNN\testing_graphs\iteration_5\model_" + str(i) + "\model_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_speed_diff.png", dpi=300)
        plt.close()




<>:44: SyntaxWarning: invalid escape sequence '\m'
<>:54: SyntaxWarning: invalid escape sequence '\m'
<>:44: SyntaxWarning: invalid escape sequence '\m'
<>:54: SyntaxWarning: invalid escape sequence '\m'
C:\Users\ROG\AppData\Local\Temp\ipykernel_24628\932876460.py:44: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(r"C:\my files\thesis\AI_training\Simple RNN\testing_graphs\iteration_5\model_" + str(i) + "\model_" + str(i) + "_" +test_data[j] + "_act_vs_predicted_speed.png", dpi=300,)
C:\Users\ROG\AppData\Local\Temp\ipykernel_24628\932876460.py:54: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(r"C:\my files\thesis\AI_training\Simple RNN\testing_graphs\iteration_5\model_" + str(i) + "\model_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_speed_diff.png", dpi=300)


In [3]:
import os

# Define the path where you want to create the directory
directory_path = r"C:\my files\thesis\AI_training\Simple RNN\testing_graphs\iteration_5\model_"

for k in range(12):
    # Create the directory
    directory = directory_path + str(k)
    os.makedirs(directory, exist_ok=True)
